# 2023 CITS4012 Assignment

**Team 33 :**

**23215183**

**23471149**

**23990486**

# Readme
---
Before running the code, make sure the path of the dataset can be reached by Google Colab.

We are using Tensorflow Keras for building the model. First, we import all of the needed modules and library.
We use Pretrained Glove Embedding glove-wiki-gigaword-100 and Cosine Product for our basic model. Basic model is also trained with 10 epochs.

This Notebook contains 3 sections: Dataset Preprocessing, Model Implementation, and Model Testing. 

In the Dataset preprocessing, we do labelling the token of each word, then perform feature extraction: TF_IDF, Word Match Feature, and POS-Tag.

In model testing we perform several different input(Glove, Glove+POS, Glove+POS+TF_IDF), attention(Dot Product, Scaled Dot Product, Cosine Product), and epoch(5,10,15,20,25). 


***The code below is not optimized***


---



#                       


---



---

# 1.DataSet Processing
(You can add as many code blocks and text blocks as you need. However, YOU SHOULD NOT MODIFY the section title) 

## Import library

In [1]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer

from sklearn.preprocessing import MultiLabelBinarizer


from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dot, Activation
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Concatenate,TimeDistributed, Bidirectional


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import random
import nltk

# Downloading required corpus
nltk.download('punkt')
nltk.download('brown')
nltk.download('averaged_perceptron_tagger')


from nltk.tokenize import word_tokenize
from nltk.corpus import brown


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [2]:
from google.colab import drive
drive.mount("/content/drive/")

Mounted at /content/drive/


In [3]:
import os
#change the direction to the path in the Google drive, for running from Google Colab, change to your own path before running
os.chdir('/content/drive/MyDrive/Colab Notebooks/NLP Assignment Submission') 

In [4]:
import os
# read TSV file as a dataframe
train_df = pd.read_csv('WikiQA-train.tsv', sep='\t')

# display the dataframe
train_df.head(2)

,QuestionID,Question,DocumentID,DocumentTitle,SentenceID,Sentence,Label
0,Q1,how are glacier caves formed?,D1,Glacier cave,D1-0,A partly submerged glacier cave on Perito More...,0
1,Q1,how are glacier caves formed?,D1,Glacier cave,D1-1,The ice facade is approximately 60 m high,0


## Data Wrangling

In [5]:
train_df['Document'] = train_df.groupby('DocumentID')['Sentence'].transform(lambda x: '.'.join(x))

train_df_1 = train_df[train_df['Label'] == 1][['Question', 'Document', 'Sentence']]

train_df_1 = train_df_1.rename(columns={'Sentence': 'Answer'})
train_df_1.reset_index(drop=True, inplace=True)
train_df_1.head()

,Question,Document,Answer
0,how are glacier caves formed?,A partly submerged glacier cave on Perito More...,A glacier cave is a cave formed within the ice...
1,how much is 1 tablespoon of water,This tablespoon has a capacity of about 15 mL....,This tablespoon has a capacity of about 15 mL.
2,how much is 1 tablespoon of water,This tablespoon has a capacity of about 15 mL....,In the USA one tablespoon (measurement unit) i...
3,how much is 1 tablespoon of water,This tablespoon has a capacity of about 15 mL....,In Australia one tablespoon (measurement unit)...
4,how much are the harry potter movies worth,Harry Potter is a series of seven fantasy nove...,The series also originated much tie-in merchan...


### Label Tokenizaion

In [6]:
def label_tokens(row):
    
    doc = row['Document']
    ans = row['Answer']
    
    # Tokenize the document
    tokens = nltk.word_tokenize(doc)
    
    # Initialize a list to hold the labels for each token
    labels = []
    
    # Loop through each token and label it
    for i in range(len(tokens)):
        token = tokens[i]
        
        if token in nltk.word_tokenize(ans):
            # Token is part of the answer
            if i == 0:
                # First token of the answer sentence
                labels.append('START')
            elif i == len(tokens) - 1:
                # Last token of the answer sentence
                labels.append('END')
            else:
                # Token is within the answer sentence
                labels.append('INNER')
        else:
            # Token is not part of the answer
            labels.append('NONE')
    
    # Add the labels to the row and return it
    row['Token_Labels'] = labels
    return row

# Apply the label_tokens function to the dataframe
df_1 = train_df_1.apply(label_tokens, axis=1)

### Feature extraction

#### TF-IDF

In [7]:
merged_ans_list = train_df['Sentence'].tolist()
corpus=merged_ans_list
len(corpus)

20347

In [8]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize, sent_tokenize

import re
import numpy as np

nltk.download('stopwords')
from nltk.corpus import stopwords as sw


merged_list_TFIDF = []
merged_tokenized_TFIDF_col = []

for i in range(len(corpus)):

  # Tokenize sentences - for only doc1
  tokenized_sentence = sent_tokenize(corpus[i])

  # Remove punctuations - for only doc1
  clean_doc1 = re.sub(r'[^\w\s]','',corpus[i])

  # Tokenize words - for only doc1
  tokenized_doc1 = word_tokenize(clean_doc1)

  # Convert the tokens into lowercase: lower_tokens
  lower_tokens = [t.lower() for t in tokenized_doc1]

  # stop word removal
  sww = sw.words()
  tokenized_doc1 = [w for w in lower_tokens if not w in sww]

  merged_list_TFIDF.extend(tokenized_doc1)
  merged_tokenized_TFIDF_col.append(tokenized_doc1)

print("\nMerged List:")
print(merged_list_TFIDF)

Output hidden; open in https://colab.research.google.com to view.

In [9]:
print("\nMerged tokenized_word (in lower case, w/o stopwords): ")
print(merged_tokenized_TFIDF_col)

Output hidden; open in https://colab.research.google.com to view.

In [10]:
train_DF = {}

for tokenized_doc in merged_tokenized_TFIDF_col:
    # get each unique word in the doc - and count the number of occurrences in the document
    for term in np.unique(tokenized_doc):
        try:
            train_DF[term] +=1
        except:
            train_DF[term] =1

train_DF

{'cave': 15,
 'glacier': 5,
 'moreno': 1,
 'partly': 15,
 'perito': 1,
 'submerged': 5,
 '60': 37,
 'approximately': 99,
 'facade': 2,
 'high': 208,
 'ice': 42,
 'formations': 6,
 'titlis': 1,
 'formed': 109,
 'bedrock': 1,
 'called': 357,
 'caves': 3,
 'contain': 60,
 'describe': 41,
 'properly': 8,
 'term': 311,
 'yearround': 6,
 'circle': 17,
 'circular': 8,
 'circumference': 2,
 'motion': 51,
 'movement': 102,
 'object': 45,
 'path': 18,
 'physics': 18,
 'rotation': 11,
 'angular': 1,
 'changing': 17,
 'constant': 24,
 'nonuniform': 1,
 'rate': 110,
 'speed': 42,
 'uniform': 16,
 'axis': 23,
 'body': 181,
 'fixed': 40,
 'involves': 39,
 'parts': 95,
 'threedimensional': 7,
 'center': 138,
 'equations': 1,
 'mass': 98,
 'artificial': 15,
 'car': 45,
 'circles': 7,
 'curve': 7,
 'earth': 143,
 'electron': 33,
 'examples': 24,
 'field': 103,
 'gear': 11,
 'height': 34,
 'include': 285,
 'inside': 47,
 'magnetic': 19,
 'mechanism': 20,
 'moving': 21,
 'orbiting': 5,
 'perpendicular': 2

In [11]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from collections import Counter
import math


train_tf_idf = {}

# total number of documents
N = len(merged_tokenized_TFIDF_col) 

doc_id = 0
# get each tokenised doc
for tokenized_doc in merged_tokenized_TFIDF_col:
    # initialise counter for the doc
    counter = Counter(tokenized_doc)
    # calculate total number of words in the doc
    total_num_words = len(tokenized_doc)    

    # get each unique word in the doc
    for term in np.unique(tokenized_doc):

        # calculate Term Frequency 
        tf = counter[term]/total_num_words
        
        # calculate Document Frequency
        df = train_DF[term]

        # calculate Inverse Document Frequency
        idf = math.log(N/(df+1))+1

        # calculate TF-IDF
        train_tf_idf[doc_id, term] = tf*idf

    doc_id += 1

train_tf_idf

{(0, 'cave'): 1.1640142910880173,
 (0, 'glacier'): 2.6082655116079567,
 (0, 'moreno'): 1.461077368470851,
 (0, 'partly'): 1.1640142910880173,
 (0, 'perito'): 1.461077368470851,
 (0, 'submerged'): 1.3041327558039784,
 (1, '60'): 1.4566205200259035,
 (1, 'approximately'): 1.2631037147735624,
 (1, 'facade'): 1.9644152942375588,
 (1, 'high'): 1.1156709015782185,
 (1, 'ice'): 1.4318977288324684,
 (2, 'cave'): 1.6296200075232243,
 (2, 'formations'): 1.794955722160118,
 (2, 'glacier'): 1.8257858581255697,
 (2, 'ice'): 1.4318977288324684,
 (2, 'titlis'): 2.0455083158591916,
 (3, 'cave'): 2.7160333458720403,
 (3, 'formed'): 1.036701399010581,
 (3, 'glacier'): 3.0429764302092828,
 (3, 'ice'): 1.19324810736039,
 (4, 'bedrock'): 0.7867339676381506,
 (4, 'called'): 0.38770429026578485,
 (4, 'caves'): 2.200244861246772,
 (4, 'contain'): 0.523831915052507,
 (4, 'describe'): 0.5525399339671181,
 (4, 'glacier'): 0.702225330048296,
 (4, 'ice'): 1.101459791409591,
 (4, 'properly'): 0.6710357063476681,
 (

In [12]:
import numpy as np
#sort the dictionary based on values
dict_exmaple = train_tf_idf
train_sorted_dict = sorted(dict_exmaple.items(), key=lambda x: x[1], reverse=True)
train_sorted_dict

[((1083, 'ibid'), 10.227541579295957),
 ((2920, 'barbiturates'), 10.227541579295957),
 ((2921, 'benzodiazepines'), 10.227541579295957),
 ((2924, 'methadone'), 10.227541579295957),
 ((2925, 'methaqualone'), 10.227541579295957),
 ((2928, 'propoxyphene'), 10.227541579295957),
 ((3996, '350px'), 10.227541579295957),
 ((5809, '300px'), 10.227541579295957),
 ((7592, '1166a'), 10.227541579295957),
 ((7604, 'sr44sr44wsr44sw'), 10.227541579295957),
 ((7606, 'sr1154'), 10.227541579295957),
 ((7607, '1107so'), 10.227541579295957),
 ((7608, '1131sop'), 10.227541579295957),
 ((7609, 'sg13'), 10.227541579295957),
 ((7610, 's76'), 10.227541579295957),
 ((9618, '19101913'), 10.227541579295957),
 ((10988, 'farro'), 10.227541579295957),
 ((10990, 'emmer'), 10.227541579295957),
 ((10991, 'einkorn'), 10.227541579295957),
 ((10992, 'kamut'), 10.227541579295957),
 ((10995, 'quinoa'), 10.227541579295957),
 ((10996, 'amaranth'), 10.227541579295957),
 ((11000, 'buckwheat'), 10.227541579295957),
 ((11007, 'popc

In [13]:
df = pd.DataFrame({'Merged Tokens': merged_tokenized_TFIDF_col})
print(df)

                                           Merged Tokens
0      [partly, submerged, glacier, cave, perito, mor...
1                 [ice, facade, approximately, 60, high]
2               [ice, formations, titlis, glacier, cave]
3            [glacier, cave, cave, formed, ice, glacier]
4      [glacier, caves, called, ice, caves, term, pro...
...                                                  ...
20342  [tenant, leaves, subsidized, project, lose, ac...
20343  [united, states, department, housing, urban, d...
20344  [restaurants, categorized, type, information, ...
20345                [federal, reserve, notes, mid1990s]
20346  [history, united, states, dollar, covers, 200,...

[20347 rows x 1 columns]


In [14]:
train_joined_df = train_df.join(df)
train_joined_df

,QuestionID,Question,DocumentID,DocumentTitle,SentenceID,Sentence,Label,Document,Merged Tokens
0,Q1,how are glacier caves formed?,D1,Glacier cave,D1-0,A partly submerged glacier cave on Perito More...,0,A partly submerged glacier cave on Perito More...,"[partly, submerged, glacier, cave, perito, mor..."
1,Q1,how are glacier caves formed?,D1,Glacier cave,D1-1,The ice facade is approximately 60 m high,0,A partly submerged glacier cave on Perito More...,"[ice, facade, approximately, 60, high]"
2,Q1,how are glacier caves formed?,D1,Glacier cave,D1-2,Ice formations in the Titlis glacier cave,0,A partly submerged glacier cave on Perito More...,"[ice, formations, titlis, glacier, cave]"
3,Q1,how are glacier caves formed?,D1,Glacier cave,D1-3,A glacier cave is a cave formed within the ice...,1,A partly submerged glacier cave on Perito More...,"[glacier, cave, cave, formed, ice, glacier]"
4,Q1,how are glacier caves formed?,D1,Glacier cave,D1-4,"Glacier caves are often called ice caves , but...",0,A partly submerged glacier cave on Perito More...,"[glacier, caves, called, ice, caves, term, pro..."
...,...,...,...,...,...,...,...,...,...
20342,Q3043,what is section eight housing,D2807,Section 8 (housing),D2807-8,A tenant who leaves a subsidized project will ...,0,Section 8 housing in the South Bronx.Section 8...,"[tenant, leaves, subsidized, project, lose, ac..."
20343,Q3043,what is section eight housing,D2807,Section 8 (housing),D2807-9,The United States Department of Housing and Ur...,0,Section 8 housing in the South Bronx.Section 8...,"[united, states, department, housing, urban, d..."
20344,Q3044,what is the main type of restaurant,D2808,Category:Types of restaurants,D2808-0,Restaurants categorized by type and informatio...,0,Restaurants categorized by type and informatio...,"[restaurants, categorized, type, information, ..."
20345,Q3046,what is us dollar worth based on,D2810,History of the United States dollar,D2810-0,U.S. Federal Reserve notes in the mid-1990s,0,U.S. Federal Reserve notes in the mid-1990s.Th...,"[federal, reserve, notes, mid1990s]"


#### Word Match Feature

In [15]:
import pandas as pd
import spacy

# Load the spaCy language model
nlp = spacy.load('en_core_web_sm')

def word_match(question, document):
    # Process the question and document with spaCy
    question_doc = nlp(question)
    document_doc = nlp(document)
    
    # Get the matched words
    matched_words = []
    
    # Option 1: Decapitalization
    for token in question_doc:
        if token.text.lower() in document_doc.text.lower():
            matched_words.append(token.text)
    
    # Option 2: Lemmatization
    for token in question_doc:
        lemma_word = token.lemma_
        if lemma_word in [token.lemma_ for token in document_doc]:
            matched_words.append(token.text)
    
    return matched_words

# Apply word_match function to the question and document columns
train_df['matched_words'] = train_df.apply(lambda row: word_match(row['Question'], row['Sentence']), axis=1)

train_df

,QuestionID,Question,DocumentID,DocumentTitle,SentenceID,Sentence,Label,Document,matched_words
0,Q1,how are glacier caves formed?,D1,Glacier cave,D1-0,A partly submerged glacier cave on Perito More...,0,A partly submerged glacier cave on Perito More...,"[glacier, glacier, caves]"
1,Q1,how are glacier caves formed?,D1,Glacier cave,D1-1,The ice facade is approximately 60 m high,0,A partly submerged glacier cave on Perito More...,[are]
2,Q1,how are glacier caves formed?,D1,Glacier cave,D1-2,Ice formations in the Titlis glacier cave,0,A partly submerged glacier cave on Perito More...,"[glacier, caves]"
3,Q1,how are glacier caves formed?,D1,Glacier cave,D1-3,A glacier cave is a cave formed within the ice...,1,A partly submerged glacier cave on Perito More...,"[glacier, formed, are, glacier, caves, formed]"
4,Q1,how are glacier caves formed?,D1,Glacier cave,D1-4,"Glacier caves are often called ice caves , but...",0,A partly submerged glacier cave on Perito More...,"[are, glacier, caves, are, caves]"
...,...,...,...,...,...,...,...,...,...
20342,Q3043,what is section eight housing,D2807,Section 8 (housing),D2807-8,A tenant who leaves a subsidized project will ...,0,Section 8 housing in the South Bronx.Section 8...,[]
20343,Q3043,what is section eight housing,D2807,Section 8 (housing),D2807-9,The United States Department of Housing and Ur...,0,Section 8 housing in the South Bronx.Section 8...,"[is, section, housing, section]"
20344,Q3044,what is the main type of restaurant,D2808,Category:Types of restaurants,D2808-0,Restaurants categorized by type and informatio...,0,Restaurants categorized by type and informatio...,"[the, type, restaurant, type, restaurant]"
20345,Q3046,what is us dollar worth based on,D2810,History of the United States dollar,D2810-0,U.S. Federal Reserve notes in the mid-1990s,0,U.S. Federal Reserve notes in the mid-1990s.Th...,[]


#### POS-Tag

In [16]:
import string
def remove_punctuation(text):
    # Remove all punctuation marks
    processed_text = text.translate(str.maketrans('', '', string.punctuation))
    return processed_text

In [55]:
train_documents = train_df_1['Document'].apply(remove_punctuation).tolist()
train_questions = train_df_1['Question'].apply(remove_punctuation).tolist()

In [56]:
train_token_docs=[]
for sentence in train_documents:
    tokens = word_tokenize(sentence)
    train_token_docs.append(tokens)

In [57]:
train_token_quest=[]
for sentence in train_questions:
    tokens = word_tokenize(sentence)
    train_token_quest.append(tokens)

In [58]:
# Import the POS tagger
from nltk.tag import pos_tag
# download the dependency and resource as required
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [59]:
tagged_quest_train = [[tag for _, tag in pos_tag(sentence)] for sentence in train_token_quest]
tagged_docs_train = [[tag for _, tag in pos_tag(sentence)] for sentence in train_token_docs]

In [22]:
train_df['tagged_docs'] = tagged_docs_train
train_df['tagged_quest'] = tagged_quest_train
# Display the updated train_df
train_df

,QuestionID,Question,DocumentID,DocumentTitle,SentenceID,Sentence,Label,Document,matched_words,tagged_docs,tagged_quest
0,Q1,how are glacier caves formed?,D1,Glacier cave,D1-0,A partly submerged glacier cave on Perito More...,0,A partly submerged glacier cave on Perito More...,"[glacier, glacier, caves]","[DT, RB, VBN, NN, NN, IN, NNP, NNP, NNP, DT, N...","[WRB, VBP, JJ, NNS, VBD]"
1,Q1,how are glacier caves formed?,D1,Glacier cave,D1-1,The ice facade is approximately 60 m high,0,A partly submerged glacier cave on Perito More...,[are],"[DT, RB, VBN, NN, NN, IN, NNP, NNP, NNP, DT, N...","[WRB, VBP, JJ, NNS, VBD]"
2,Q1,how are glacier caves formed?,D1,Glacier cave,D1-2,Ice formations in the Titlis glacier cave,0,A partly submerged glacier cave on Perito More...,"[glacier, caves]","[DT, RB, VBN, NN, NN, IN, NNP, NNP, NNP, DT, N...","[WRB, VBP, JJ, NNS, VBD]"
3,Q1,how are glacier caves formed?,D1,Glacier cave,D1-3,A glacier cave is a cave formed within the ice...,1,A partly submerged glacier cave on Perito More...,"[glacier, formed, are, glacier, caves, formed]","[DT, RB, VBN, NN, NN, IN, NNP, NNP, NNP, DT, N...","[WRB, VBP, JJ, NNS, VBD]"
4,Q1,how are glacier caves formed?,D1,Glacier cave,D1-4,"Glacier caves are often called ice caves , but...",0,A partly submerged glacier cave on Perito More...,"[are, glacier, caves, are, caves]","[DT, RB, VBN, NN, NN, IN, NNP, NNP, NNP, DT, N...","[WRB, VBP, JJ, NNS, VBD]"
...,...,...,...,...,...,...,...,...,...,...,...
20342,Q3043,what is section eight housing,D2807,Section 8 (housing),D2807-8,A tenant who leaves a subsidized project will ...,0,Section 8 housing in the South Bronx.Section 8...,[],"[NN, CD, NN, IN, DT, NNP, NNP, CD, IN, DT, NNP...","[WP, VBZ, NN, CD, NN]"
20343,Q3043,what is section eight housing,D2807,Section 8 (housing),D2807-9,The United States Department of Housing and Ur...,0,Section 8 housing in the South Bronx.Section 8...,"[is, section, housing, section]","[NN, CD, NN, IN, DT, NNP, NNP, CD, IN, DT, NNP...","[WP, VBZ, NN, CD, NN]"
20344,Q3044,what is the main type of restaurant,D2808,Category:Types of restaurants,D2808-0,Restaurants categorized by type and informatio...,0,Restaurants categorized by type and informatio...,"[the, type, restaurant, type, restaurant]","[NNS, VBN, IN, NN, CC, NN, IN, DT, JJ, NNS]","[WP, VBZ, DT, JJ, NN, IN, NN]"
20345,Q3046,what is us dollar worth based on,D2810,History of the United States dollar,D2810-0,U.S. Federal Reserve notes in the mid-1990s,0,U.S. Federal Reserve notes in the mid-1990s.Th...,[],"[NNP, NNP, NNP, NNS, IN, DT, JJ, NN, IN, DT, N...","[WP, VBZ, PRP, NN, NN, VBN, IN]"


## Test set

In [23]:
test_df = pd.read_csv('WikiQA-test.tsv', sep='\t')

### Data Wrangling

In [24]:
test_df['Document'] = test_df.groupby('DocumentID')['Sentence'].transform(lambda x: '.'.join(x))

test_df_1 = test_df[test_df['Label'] == 1][['Question', 'Document', 'Sentence']]

test_df_1 = test_df_1.rename(columns={'Sentence': 'Answer'})
test_df_1.reset_index(drop=True, inplace=True)
test_df_1.head()

,Question,Document,Answer
0,HOW AFRICAN AMERICANS WERE IMMIGRATED TO THE US,African immigration to the United States refer...,"As such, African immigrants are to be distingu..."
1,how a water pump works,"A small, electrically powered pump.A large, el...",Pumps operate by some mechanism (typically rec...
2,how old was sue lyon when she made lolita,Lolita is a 1962 comedy-drama film by Stanley ...,"The actress who played Lolita, Sue Lyon , was ..."
3,how are antibodies used in,Each antibody binds to a specific antigen ; an...,"An antibody (Ab), also known as an immunoglobu..."
4,how are antibodies used in,Each antibody binds to a specific antigen ; an...,The antibody recognizes a unique part of the f...


#### Label Tokenization

In [25]:
def label_tokens(row):
    
    doc = row['Document']
    ans = row['Answer']
    
    # Tokenize the document
    tokens = nltk.word_tokenize(doc)
    
    # Initialize a list to hold the labels for each token
    labels = []
    
    # Loop through each token and label it
    for i in range(len(tokens)):
        token = tokens[i]
        
        if token in nltk.word_tokenize(ans):
            # Token is part of the answer
            if i == 0:
                # First token of the answer sentence
                labels.append('START')
            elif i == len(tokens) - 1:
                # Last token of the answer sentence
                labels.append('END')
            else:
                # Token is within the answer sentence
                labels.append('INNER')
        else:
            # Token is not part of the answer
            labels.append('NONE')
    
    # Add the labels to the row and return it
    row['Token_Labels'] = labels
    return row

# Apply the label_tokens function to the dataframe
df_2 = test_df_1.apply(label_tokens, axis=1)

In [26]:
test_documents = df_2['Document'].tolist()
test_questions = df_2['Question'].tolist()
test_answers = df_2['Token_Labels'].tolist()

# Tokenize the documents and questions
tokenizer = Tokenizer()
tokenizer.fit_on_texts(test_documents + test_questions)
num_words = len(tokenizer.word_index) + 1

# Convert documents and questions to sequences
test_document_sequences = tokenizer.texts_to_sequences(test_documents)
test_question_sequences = tokenizer.texts_to_sequences(test_questions)

# Pad the sequences to a maximum length of 100
max_sequence_length = 100
test_document_sequences_padded = pad_sequences(test_document_sequences, maxlen=max_sequence_length)
test_question_sequences_padded = pad_sequences(test_question_sequences, maxlen=max_sequence_length)


# Prepare the answer labels
test_answer_labels = []
for answer in test_answers:
    labels_encoded = []
    for token in answer:
        if token == 'START':
            labels_encoded.append(0)
        elif token == 'END':
            labels_encoded.append(1)
        elif token == 'INNER':
            labels_encoded.append(2)
        else:
            labels_encoded.append(3)
    test_answer_labels.append(labels_encoded)

# Pad the answer labels to a maximum length of 100
test_answer_labels_padded = pad_sequences(test_answer_labels, maxlen=max_sequence_length, padding='post')

# Convert answer labels to one-hot encoding
y_test_labels = to_categorical(test_answer_labels_padded, num_classes=4)

# Assign the processed data to x_test_questions, x_test_documents, y_test_labels
x_test_questions = test_question_sequences_padded
x_test_documents = test_document_sequences_padded

# Print the processed data
print("x_test_questions:", x_test_questions.shape)
print("x_test_documents:", x_test_documents.shape)
print("y_test_labels:", y_test_labels.shape)

x_test_questions: (291, 100)
x_test_documents: (291, 100)
y_test_labels: (291, 100, 4)


### Feature extraction

#### TF-IDF

In [27]:
merged_ans_list_test = test_df['Sentence'].tolist()
corpus_test=merged_ans_list_test
len(corpus_test)

6116

In [28]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize, sent_tokenize

import re
import numpy as np

nltk.download('stopwords')
from nltk.corpus import stopwords as sw


merged_list_TFIDF_test = []
merged_tokenized_TFIDF_col_test = []

for i in range(len(corpus_test)):

  # Tokenize sentences - for only doc1
  tokenized_sentence = sent_tokenize(corpus_test[i])

  # Remove punctuations - for only doc1
  clean_doc1 = re.sub(r'[^\w\s]','',corpus_test[i])

  # Tokenize words - for only doc1
  tokenized_doc1 = word_tokenize(clean_doc1)

  # Convert the tokens into lowercase: lower_tokens
  lower_tokens = [t.lower() for t in tokenized_doc1]

  # stop word removal
  sww = sw.words()
  tokenized_doc1 = [w for w in lower_tokens if not w in sww]

  merged_list_TFIDF_test.extend(tokenized_doc1)
  merged_tokenized_TFIDF_col_test.append(tokenized_doc1)

print("\nMerged List:")
print(merged_list_TFIDF_test)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!



Merged List:
['african', 'immigration', 'united', 'states', 'refers', 'immigrants', 'united', 'states', 'nationals', 'africa', 'term', 'african', 'scope', 'article', 'refers', 'geographical', 'national', 'origins', 'racial', 'affiliation', 'immigration', 'nationality', 'act', '1965', '2007', 'estimated', 'total', '08', '09', 'million', 'africans', 'immigrated', 'united', 'states', 'accounting', 'roughly', '33', 'total', 'immigration', 'united', 'states', 'period', 'african', 'immigrants', 'united', 'states', 'regions', 'africa', 'constitute', 'homogeneous', 'group', 'include', 'different', 'national', 'linguistic', 'ethnic', 'racial', 'cultural', 'social', 'backgrounds', 'african', 'immigrants', 'distinguished', 'african', 'american', 'latter', 'descendants', 'west', 'central', 'africans', 'involuntarily', 'brought', 'united', 'states', 'means', 'historic', 'atlantic', 'slave', 'trade', 'prison', 'french', 'prisoun', 'gaol', 'jail', 'place', 'physically', 'confined', 'deprived', 'rang

In [29]:
print("\nMerged tokenized_word (in lower case, w/o stopwords): ")
print(merged_tokenized_TFIDF_col_test)


Merged tokenized_word (in lower case, w/o stopwords): 
[['african', 'immigration', 'united', 'states', 'refers', 'immigrants', 'united', 'states', 'nationals', 'africa'], ['term', 'african', 'scope', 'article', 'refers', 'geographical', 'national', 'origins', 'racial', 'affiliation'], ['immigration', 'nationality', 'act', '1965', '2007', 'estimated', 'total', '08', '09', 'million', 'africans', 'immigrated', 'united', 'states', 'accounting', 'roughly', '33', 'total', 'immigration', 'united', 'states', 'period'], ['african', 'immigrants', 'united', 'states', 'regions', 'africa', 'constitute', 'homogeneous', 'group'], ['include', 'different', 'national', 'linguistic', 'ethnic', 'racial', 'cultural', 'social', 'backgrounds'], ['african', 'immigrants', 'distinguished', 'african', 'american', 'latter', 'descendants', 'west', 'central', 'africans', 'involuntarily', 'brought', 'united', 'states', 'means', 'historic', 'atlantic', 'slave', 'trade'], ['prison', 'french', 'prisoun', 'gaol', 'jail

In [30]:
test_DF = {}

for tokenized_doc in merged_tokenized_TFIDF_col_test:
    # get each unique word in the doc - and count the number of occurrences in the document
    for term in np.unique(tokenized_doc):
        try:
            test_DF[term] +=1
        except:
            test_DF[term] =1

test_DF

{'africa': 30,
 'african': 30,
 'immigrants': 10,
 'immigration': 13,
 'nationals': 1,
 'refers': 38,
 'states': 378,
 'united': 363,
 'affiliation': 3,
 'article': 16,
 'geographical': 4,
 'national': 131,
 'origins': 4,
 'racial': 6,
 'scope': 3,
 'term': 107,
 '08': 2,
 '09': 1,
 '1965': 17,
 '2007': 28,
 '33': 3,
 'accounting': 4,
 'act': 43,
 'africans': 5,
 'estimated': 18,
 'immigrated': 1,
 'million': 90,
 'nationality': 2,
 'period': 41,
 'roughly': 9,
 'total': 58,
 'constitute': 3,
 'group': 43,
 'homogeneous': 1,
 'regions': 15,
 'backgrounds': 2,
 'cultural': 29,
 'different': 63,
 'ethnic': 7,
 'include': 89,
 'linguistic': 2,
 'social': 46,
 'american': 255,
 'atlantic': 14,
 'brought': 16,
 'central': 43,
 'descendants': 5,
 'distinguished': 9,
 'historic': 9,
 'involuntarily': 1,
 'latter': 18,
 'means': 22,
 'slave': 11,
 'trade': 24,
 'west': 64,
 'confined': 3,
 'deprived': 1,
 'freedoms': 2,
 'french': 41,
 'gaol': 2,
 'jail': 4,
 'personal': 18,
 'physically': 3,


In [31]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from collections import Counter
import math


tf_idf_test = {}

# total number of documents
N = len(merged_tokenized_TFIDF_col_test) 

doc_id = 0
# get each tokenised doc
for tokenized_doc in merged_tokenized_TFIDF_col_test:
    # initialise counter for the doc
    counter = Counter(tokenized_doc)
    # calculate total number of words in the doc
    total_num_words = len(tokenized_doc)    

    # get each unique word in the doc
    for term in np.unique(tokenized_doc):

        # calculate Term Frequency 
        tf = counter[term]/total_num_words
        
        # calculate Document Frequency
        df = test_DF[term]

        # calculate Inverse Document Frequency
        idf = math.log(N/(df+1))+1

        # calculate TF-IDF
        tf_idf_test[doc_id, term] = tf*idf

    doc_id += 1

tf_idf_test

{(0, 'africa'): 0.6284676362563807,
 (0, 'african'): 0.6284676362563807,
 (0, 'immigrants'): 0.7320768294250583,
 (0, 'immigration'): 0.7079606237433694,
 (0, 'nationals'): 0.9025516386489008,
 (0, 'refers'): 0.6055101920919307,
 (0, 'states'): 0.7562254723933054,
 (0, 'united'): 0.7643019398824425,
 (1, 'affiliation'): 0.8332369205929062,
 (1, 'african'): 0.6284676362563807,
 (1, 'article'): 0.6885450222992737,
 (1, 'geographical'): 0.8109225654614853,
 (1, 'national'): 0.48358616444625824,
 (1, 'origins'): 0.8109225654614853,
 (1, 'racial'): 0.777275341799364,
 (1, 'refers'): 0.6055101920919307,
 (1, 'scope'): 0.8332369205929062,
 (1, 'term'): 0.5036532339924733,
 (2, '08'): 0.3918205126536746,
 (2, '09'): 0.41025074484040946,
 (2, '1965'): 0.31037690041603583,
 (2, '2007'): 0.28869853350283997,
 (2, '33'): 0.3787440548149574,
 (2, 'accounting'): 0.36860116611885696,
 (2, 'act'): 0.26974881514230414,
 (2, 'africans'): 0.36031382262822265,
 (2, 'estimated'): 0.30791929944920515,
 (2, 

In [32]:
import numpy as np
#sort the dictionary based on values
dict_exmaple = tf_idf_test
sorted_dict = sorted(dict_exmaple.items(), key=lambda x: x[1], reverse=True)
sorted_dict

[((1160, 'barbiturates'), 9.025516386489008),
 ((1161, 'benzodiazepines'), 9.025516386489008),
 ((1165, 'methaqualone'), 9.025516386489008),
 ((1168, 'propoxyphene'), 9.025516386489008),
 ((2586, 'framelessmaster'), 9.025516386489008),
 ((2778, 'christadelphians'), 9.025516386489008),
 ((3546,
   '1000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000'),
  9.025516386489008),
 ((3560, '69'), 9.025516386489008),
 ((3907, 'axon2'), 9.025516386489008),
 ((3910, 'myofibril'), 9.025516386489008),
 ((3913, 'sarcolemma3'), 9.025516386489008),
 ((3916, 'mitochondrion'), 9.025516386489008),
 ((222, 'lactose'), 8.620051278380842),
 ((955, '200px'), 8.620051278380842),
 ((1164, 'methadone'), 8.620051278380842),
 ((1754, '200px'), 8.620051278380842),
 ((2040, 'fool'), 8.620051278380842),
 ((6087, 'beetles'), 8.620051278380842),
 ((294, 'thumb'), 8.332369205929062),
 ((1163, 'cocaine'), 8.332369205929062),
 ((4494, 'sauce'), 8.109225654614853),
 ((22

In [33]:
df_test = pd.DataFrame({'Merged Tokens': merged_tokenized_TFIDF_col_test})
print(df_test)

                                          Merged Tokens
0     [african, immigration, united, states, refers,...
1     [term, african, scope, article, refers, geogra...
2     [immigration, nationality, act, 1965, 2007, es...
3     [african, immigrants, united, states, regions,...
4     [include, different, national, linguistic, eth...
...                                                 ...
6111  [word, female, equine, animals, particularly, ...
6112                        [broodmare, mare, breeding]
6113                      [horses, female, parent, dam]
6114  [adult, male, horse, called, stallion, castrat...
6115  [occasionally, term, horse, restrictive, desig...

[6116 rows x 1 columns]


In [34]:
test_joined_df = test_df.join(df_test)
test_joined_df

,QuestionID,Question,DocumentID,DocumentTitle,SentenceID,Sentence,Label,Document,Merged Tokens
0,Q0,HOW AFRICAN AMERICANS WERE IMMIGRATED TO THE US,D0,African immigration to the United States,D0-0,African immigration to the United States refer...,0,African immigration to the United States refer...,"[african, immigration, united, states, refers,..."
1,Q0,HOW AFRICAN AMERICANS WERE IMMIGRATED TO THE US,D0,African immigration to the United States,D0-1,The term African in the scope of this article ...,0,African immigration to the United States refer...,"[term, african, scope, article, refers, geogra..."
2,Q0,HOW AFRICAN AMERICANS WERE IMMIGRATED TO THE US,D0,African immigration to the United States,D0-2,From the Immigration and Nationality Act of 19...,0,African immigration to the United States refer...,"[immigration, nationality, act, 1965, 2007, es..."
3,Q0,HOW AFRICAN AMERICANS WERE IMMIGRATED TO THE US,D0,African immigration to the United States,D0-3,African immigrants in the United States come f...,0,African immigration to the United States refer...,"[african, immigrants, united, states, regions,..."
4,Q0,HOW AFRICAN AMERICANS WERE IMMIGRATED TO THE US,D0,African immigration to the United States,D0-4,"They include people from different national, l...",0,African immigration to the United States refer...,"[include, different, national, linguistic, eth..."
...,...,...,...,...,...,...,...,...,...
6111,Q3045,what is an open mare?,D2809,Mare,D2809-4,The word can also be used for other female equ...,0,A broodmare and foal.A mare is an adult female...,"[word, female, equine, animals, particularly, ..."
6112,Q3045,what is an open mare?,D2809,Mare,D2809-5,A broodmare is a mare used for breeding.,0,A broodmare and foal.A mare is an adult female...,"[broodmare, mare, breeding]"
6113,Q3045,what is an open mare?,D2809,Mare,D2809-6,A horse's female parent is known as its dam.,0,A broodmare and foal.A mare is an adult female...,"[horses, female, parent, dam]"
6114,Q3045,what is an open mare?,D2809,Mare,D2809-7,"An adult male horse is called a stallion , or,...",0,A broodmare and foal.A mare is an adult female...,"[adult, male, horse, called, stallion, castrat..."


#### Word Match Feature

In [35]:
import pandas as pd
import spacy

# Load the spaCy language model
nlp = spacy.load('en_core_web_sm')

def word_match(question, document):
    # Process the question and document with spaCy
    question_doc = nlp(question)
    document_doc = nlp(document)
    
    # Get the matched words
    matched_words = []
    
    # Option 1: Decapitalization
    for token in question_doc:
        if token.text.lower() in document_doc.text.lower():
            matched_words.append(token.text)
    
    # Option 2: Lemmatization
    for token in question_doc:
        lemma_word = token.lemma_
        if lemma_word in [token.lemma_ for token in document_doc]:
            matched_words.append(token.text)
    
    return matched_words

# Apply word_match function to the question and document columns
test_df['matched_words'] = test_df.apply(lambda row: word_match(row['Question'], row['Sentence']), axis=1)

test_df

,QuestionID,Question,DocumentID,DocumentTitle,SentenceID,Sentence,Label,Document,matched_words
0,Q0,HOW AFRICAN AMERICANS WERE IMMIGRATED TO THE US,D0,African immigration to the United States,D0-0,African immigration to the United States refer...,0,African immigration to the United States refer...,"[AFRICAN, WERE, TO, THE, WERE, TO, THE]"
1,Q0,HOW AFRICAN AMERICANS WERE IMMIGRATED TO THE US,D0,African immigration to the United States,D0-1,The term African in the scope of this article ...,0,African immigration to the United States refer...,"[AFRICAN, TO, THE, TO, THE]"
2,Q0,HOW AFRICAN AMERICANS WERE IMMIGRATED TO THE US,D0,African immigration to the United States,D0-2,From the Immigration and Nationality Act of 19...,0,African immigration to the United States refer...,"[AFRICAN, IMMIGRATED, TO, THE, IMMIGRATED, TO,..."
3,Q0,HOW AFRICAN AMERICANS WERE IMMIGRATED TO THE US,D0,African immigration to the United States,D0-3,African immigrants in the United States come f...,0,African immigration to the United States refer...,"[AFRICAN, THE, US, THE]"
4,Q0,HOW AFRICAN AMERICANS WERE IMMIGRATED TO THE US,D0,African immigration to the United States,D0-4,"They include people from different national, l...",0,African immigration to the United States refer...,[THE]
...,...,...,...,...,...,...,...,...,...
6111,Q3045,what is an open mare?,D2809,Mare,D2809-4,The word can also be used for other female equ...,0,A broodmare and foal.A mare is an adult female...,"[is, an, is]"
6112,Q3045,what is an open mare?,D2809,Mare,D2809-5,A broodmare is a mare used for breeding.,0,A broodmare and foal.A mare is an adult female...,"[is, mare, is, mare]"
6113,Q3045,what is an open mare?,D2809,Mare,D2809-6,A horse's female parent is known as its dam.,0,A broodmare and foal.A mare is an adult female...,"[is, is]"
6114,Q3045,what is an open mare?,D2809,Mare,D2809-7,"An adult male horse is called a stallion , or,...",0,A broodmare and foal.A mare is an adult female...,"[is, an, is, an]"


#### POS-Tag

In [36]:
import string
def remove_punctuation(text):
    # Remove all punctuation marks
    processed_text = text.translate(str.maketrans('', '', string.punctuation))
    return processed_text

In [37]:
test_documents = test_df_1['Document'].apply(remove_punctuation).tolist()
test_questions = test_df_1['Question'].apply(remove_punctuation).tolist()

In [38]:
test_token_docs=[]
for sentence in test_documents:
    tokens = word_tokenize(sentence)
    test_token_docs.append(tokens)

In [39]:
test_token_quest=[]
for sentence in test_questions:
    tokens = word_tokenize(sentence)
    test_token_quest.append(tokens)

In [40]:
test_tagged_quest = [[tag for _, tag in pos_tag(sentence)] for sentence in test_token_quest]
test_tagged_docs = [[tag for _, tag in pos_tag(sentence)] for sentence in test_token_docs]

In [41]:
test_df_1['tagged_docs'] = test_tagged_docs
test_df_1['tagged_quest'] = test_tagged_quest
# Display the updated test_df
test_df_1

,Question,Document,Answer,tagged_docs,tagged_quest
0,HOW AFRICAN AMERICANS WERE IMMIGRATED TO THE US,African immigration to the United States refer...,"As such, African immigrants are to be distingu...","[JJ, NN, TO, DT, NNP, NNPS, NNS, TO, NNS, TO, ...","[WRB, JJ, NNPS, NNP, NNP, NNP, NNP, NNP]"
1,how a water pump works,"A small, electrically powered pump.A large, el...",Pumps operate by some mechanism (typically rec...,"[DT, JJ, RB, VBN, JJ, JJ, RB, JJ, NN, NN, IN, ...","[WRB, DT, NN, NN, NNS]"
2,how old was sue lyon when she made lolita,Lolita is a 1962 comedy-drama film by Stanley ...,"The actress who played Lolita, Sue Lyon , was ...","[NNP, VBZ, DT, CD, NN, NN, IN, NNP, NNP, VBN, ...","[WRB, JJ, VBD, NN, NN, WRB, PRP, VBD, NN]"
3,how are antibodies used in,Each antibody binds to a specific antigen ; an...,"An antibody (Ab), also known as an immunoglobu...","[DT, NN, VBZ, TO, DT, JJ, NN, DT, NN, JJ, TO, ...","[WRB, VBP, NNS, VBN, IN]"
4,how are antibodies used in,Each antibody binds to a specific antigen ; an...,The antibody recognizes a unique part of the f...,"[DT, NN, VBZ, TO, DT, JJ, NN, DT, NN, JJ, TO, ...","[WRB, VBP, NNS, VBN, IN]"
...,...,...,...,...,...
286,what is the main component of vaccines,Jonas Salk in 1955 holds two bottles of a cult...,The agent stimulates the body's immune system ...,"[NNP, NNP, IN, CD, VBZ, CD, NNS, IN, DT, NN, V...","[WP, VBZ, DT, JJ, NN, IN, NNS]"
287,what is preciosa crystal?,Preciosa is the luxury brand name for the rang...,Preciosa is the luxury brand name for the rang...,"[NNP, VBZ, DT, NN, NN, NN, IN, DT, NN, IN, JJ,...","[WP, VBZ, JJ, NN]"
288,who are all of the jonas brothers,The Jonas Brothers are an American pop rock ba...,"Formed in 2005, they have gained popularity fr...","[DT, NNP, NNPS, VBP, DT, JJ, NN, NN, VBN, IN, ...","[WP, VBP, DT, IN, DT, NN, NNS]"
289,who is mary matalin married to,This is about the political professional..For ...,She is married to Democratic political consult...,"[DT, VBZ, IN, DT, JJ, NN, DT, NN, VBP, NNP, NN...","[WP, VBZ, JJ, NN, VBD, TO]"


# 2.QA Model Implementation
(You can add as many code blocks and text blocks as you need. However, YOU SHOULD NOT MODIFY the section title) 

## Illustration of Data preparation before building a model

In [42]:
train_documents = df_1['Document'].tolist()
train_questions = df_1['Question'].tolist()
train_answers = df_1['Token_Labels'].tolist()

# Tokenize the documents and questions
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_documents + train_questions)
num_words = len(tokenizer.word_index) + 1

# Convert documents and questions to sequences
train_document_sequences = tokenizer.texts_to_sequences(train_documents)
train_question_sequences = tokenizer.texts_to_sequences(train_questions)

# Pad the sequences to a maximum length of 100 
# make all the sequences to same length
max_sequence_length = 100
train_document_sequences_padded = pad_sequences(train_document_sequences, maxlen=max_sequence_length)
train_question_sequences_padded = pad_sequences(train_question_sequences, maxlen=max_sequence_length)


# Prepare the answer labels
train_answer_labels = []
for answer in train_answers:
    labels_encoded = []
    for token in answer:
        if token == 'START':
            labels_encoded.append(0)
        elif token == 'END':
            labels_encoded.append(1)
        elif token == 'INNER':
            labels_encoded.append(2)
        else:
            labels_encoded.append(3)
    train_answer_labels.append(labels_encoded)

# Pad the answer labels to a maximum length of 100
train_answer_labels_padded = pad_sequences(train_answer_labels, maxlen=max_sequence_length, padding='post')

# Convert answer labels to one-hot encoding
y_train_labels = to_categorical(train_answer_labels_padded, num_classes=4)

# Assign the processed data to x_train_questions, x_train_documents, y_train_labels
x_train_questions = train_question_sequences_padded
x_train_documents = train_document_sequences_padded

# Print the processed data
print("x_train_questions:", x_train_questions.shape)
print("x_train_documents:", x_train_documents.shape)
print("y_train_labels:", y_train_labels.shape)

x_train_questions: (1039, 100)
x_train_documents: (1039, 100)
y_train_labels: (1039, 100, 4)


## Vocab size
As a thumb rule it should be more than the index

In [43]:
print("x_train_questions - Min value:", np.min(x_train_questions))
print("x_train_questions - Max value:", np.max(x_train_questions))
print("x_train_documents - Min value:", np.min(x_train_documents))
print("x_train_documents - Max value:", np.max(x_train_documents))

x_train_questions - Min value: 0
x_train_questions - Max value: 20756
x_train_documents - Min value: 0
x_train_documents - Max value: 20681


## Pretrained Glove Word Embedding
***glove-wiki-gigaword-100 is used***

In [44]:
from tensorflow import keras
from tensorflow.keras.layers import LSTM, Dense, Embedding, TimeDistributed, Dot, Activation, Concatenate, Reshape, Lambda
import tensorflow.keras.backend as K
import numpy as np
import gensim.downloader as api

# Define the maximum length of the input sequences
max_sequence_length = 100

# Load pre-trained GloVe embeddings
embedding_name = 'glove-wiki-gigaword-100'
embedding_model = api.load(embedding_name)

# Get vocabulary size and embedding dimension from the GloVe model
vocabulary_size_glv = len(embedding_model.key_to_index) + 1  # Add 1 for the unknown word
embedding_dim_glv = embedding_model.vector_size

# Create an embedding matrix
embedding_matrix_glv = np.zeros((vocabulary_size_glv, embedding_dim_glv))
for word, index in embedding_model.key_to_index.items():
    embedding_matrix_glv[index] = embedding_model.get_vector(word)

# Define the embedding layer using the pre-trained embeddings
embedding_layer_glv = Embedding(vocabulary_size_glv, embedding_dim_glv,
                                weights=[embedding_matrix_glv],
                                trainable=False)

# Define the inputs
question_input = keras.Input(shape=(max_sequence_length,), name='question_input')
document_input = keras.Input(shape=(max_sequence_length,), name='document_input')

# Replace the embedding layers in the original code with the pre-trained embedding layer
question_embedding = embedding_layer_glv(question_input)
document_embedding = embedding_layer_glv(document_input)

# Define the number of hidden units in the LSTM layers
lstm_units = 128

# Apply LSTM layers to the embeddings
question_lstm = Bidirectional(LSTM(lstm_units, return_sequences=True))(question_embedding)
question_lstm = Bidirectional(LSTM(lstm_units, return_sequences=True))(question_lstm)
document_lstm = Bidirectional(LSTM(lstm_units, return_sequences=True))(document_embedding)
document_lstm = Bidirectional(LSTM(lstm_units, return_sequences=True))(document_lstm)

# Attention variants
def dot_product_attention(inputs):
    # Compute dot product attention
    q, d = inputs
    attention_weights = Dot(axes=(2, 2))([q, d])
    attention_weights = Activation('softmax')(attention_weights)
    return attention_weights

def scaled_dot_product_attention(inputs):
    # Compute scaled dot product attention
    q, d = inputs
    attention_weights = Dot(axes=(2, 2))([q, d])
    attention_weights = Lambda(lambda x: x / K.sqrt(K.cast(K.shape(x)[-1], dtype='float32')))(attention_weights)
    attention_weights = Activation('softmax')(attention_weights)
    return attention_weights

def cosine_product_attention(inputs):
    # Compute cosine product attention
    q, d = inputs
    q = K.l2_normalize(q, axis=-1)
    d = K.l2_normalize(d, axis=-1)
    attention_weights = Dot(axes=(2, 2))([q, d])
    attention_weights = Activation('softmax')(attention_weights)
    return attention_weights

# Apply attention mechanism to the document LSTM output
#attention = dot_product_attention([question_lstm, document_lstm])
#attention = scaled_dot_product_attention([question_lstm, document_lstm])
attention = cosine_product_attention([question_lstm, document_lstm])


# Reshape attention weights to match LSTM output shape
attention = Reshape((-1, max_sequence_length))(attention)

# Permute attention weights to match LSTM output shape
attention = Lambda(lambda x: K.permute_dimensions(x, (0, 2, 1)))(attention)

# Apply attention weights to LSTM output
weighted_representation = Dot(axes=(2, 1))([attention, document_lstm])

# Concatenate attention-weighted representation with LSTM output
concatenated_representation = Concatenate()([document_lstm, weighted_representation])

# Output layer for token prediction
output = TimeDistributed(Dense(4, activation='softmax'))(concatenated_representation)

# Create the model
model = keras.Model(inputs=[question_input, document_input], outputs=output)

# Define the loss function
loss_fn = keras.losses.CategoricalCrossentropy()

# Compile the model
model.compile(optimizer='adam', loss=loss_fn, metrics=['accuracy'])

# Print the model summary
model.summary()

# Train the model
model.fit(x=[x_train_questions, x_train_documents], y=y_train_labels, epochs=10)


[==================================================] 100.0% 128.1/128.1MB downloaded
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 document_input (InputLayer)    [(None, 100)]        0           []                               
                                                                                                  
 question_input (InputLayer)    [(None, 100)]        0           []                               
                                                                                                  
 embedding (Embedding)          (None, 100, 100)     40000100    ['question_input[0][0]',         
                                                                  'document_input[0][0]']         
                                                                                                  
 bidirect

In [45]:
from tensorflow.keras.models import save_model

# Save the model to a specific directory
model.save('model_glove_epoch10_cosattn.h5')

# 3.Model Testing
(You can add as many code blocks and text blocks as you need. However, YOU SHOULD NOT MODIFY the section title) 

In [46]:
# Evaluate the performance of the model
# Assuming you have ground truth labels for the test data as y_test_labels
loss, accuracy = model.evaluate([x_test_questions, x_test_documents], y_test_labels)
print('\n')
print("Test loss:", loss)
print("Test accuracy:", accuracy)


10/10 [==============================] - 8s 486ms/step - loss: 0.6057 - accuracy: 0.7099


Test loss: 0.6056978702545166
Test accuracy: 0.7098969221115112


In [47]:
import numpy as np
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, classification_report
from tensorflow.keras.models import load_model


# Make predictions
y_pred = model.predict([x_test_questions, x_test_documents])

# Convert predicted labels to class indices (assuming one-hot encoding)
y_pred_indices = np.argmax(y_pred, axis=-1)
y_true_indices = np.argmax(y_test_labels, axis=-1)

# Flatten the indices for computing metrics
y_pred_indices_flat = y_pred_indices.flatten()
y_true_indices_flat = y_true_indices.flatten()


print(classification_report(y_pred_indices_flat,y_true_indices_flat))

# Compute precision, recall, and F1-score
precision_cosattn = precision_score(y_true_indices_flat, y_pred_indices_flat, average='weighted')
recall_cosattn = recall_score(y_true_indices_flat, y_pred_indices_flat, average='weighted')
f1_cosattn = f1_score(y_true_indices_flat, y_pred_indices_flat, average='weighted')

 #Print the metrics
print('Precision:', precision_cosattn)
print('Recall:', recall_cosattn)
print('F1-score:', f1_cosattn)


10/10 [==============================] - 9s 366ms/step
              precision    recall  f1-score   support

           0       0.98      0.88      0.92      2201
           1       0.80      0.95      0.87       237
           2       0.08      0.75      0.14       894
           3       0.98      0.69      0.81     25768

    accuracy                           0.71     29100
   macro avg       0.71      0.81      0.69     29100
weighted avg       0.95      0.71      0.80     29100

Precision: 0.7234280872385328
Recall: 0.7098969072164949
F1-score: 0.620341837358706


###3.1. Input Embedding Ablation Study

(You can add as many code blocks and text blocks as you need. However, YOU SHOULD NOT MODIFY the section title) 



#### Glove+POS

In [61]:
#Convert Pos tah to sequence
tag_doc_sequences = tokenizer.texts_to_sequences(tagged_docs_train)
tag_quest_sequences = tokenizer.texts_to_sequences(tagged_quest_train)

# Pad the sequences to a maximum length of 100 
# make all the sequences to same length
max_sequence_length = 100
train_tag_doc_sequences_padded = pad_sequences(tag_doc_sequences, maxlen=max_sequence_length)
train_tag_quest_sequences_padded = pad_sequences(tag_quest_sequences, maxlen=max_sequence_length)

(20347,)

In [62]:
#concat Pretrained Glove Word Embedding with Pos_tag
train_concated_docs = train_tag_doc_sequences_padded + x_train_documents
train_concated_quest = train_tag_doc_sequences_padded + x_train_questions

In [63]:
# Train the model
model.fit(x=[train_concated_quest, train_concated_docs], y=y_train_labels, epochs=10)

Epoch 1/10
33/33 [==============================] - 57s 2s/step - loss: 0.5954 - accuracy: 0.7146
Epoch 2/10
33/33 [==============================] - 56s 2s/step - loss: 0.5871 - accuracy: 0.7149
Epoch 3/10
33/33 [==============================] - 55s 2s/step - loss: 0.5904 - accuracy: 0.7176
Epoch 4/10
33/33 [==============================] - 52s 2s/step - loss: 0.5830 - accuracy: 0.7166
Epoch 5/10
33/33 [==============================] - 54s 2s/step - loss: 0.5824 - accuracy: 0.7189
Epoch 6/10
33/33 [==============================] - 53s 2s/step - loss: 0.5775 - accuracy: 0.7169
Epoch 7/10
33/33 [==============================] - 54s 2s/step - loss: 0.5722 - accuracy: 0.7209
Epoch 8/10
33/33 [==============================] - 53s 2s/step - loss: 0.5699 - accuracy: 0.7211
Epoch 9/10
33/33 [==============================] - 54s 2s/step - loss: 0.5720 - accuracy: 0.7201
Epoch 10/10
33/33 [==============================] - 54s 2s/step - loss: 0.5706 - accuracy: 0.7225


In [64]:
#make Pos Tag of test data to sequence

tag_doc_sequences = tokenizer.texts_to_sequences(test_tagged_docs)
tag_quest_sequences = tokenizer.texts_to_sequences(test_tagged_quest)

# Pad the sequences to a maximum length of 100 
# make all the sequences to same length
test_tag_doc_sequences_padded = pad_sequences(tag_doc_sequences, maxlen=max_sequence_length)
test_tag_quest_sequences_padded = pad_sequences(tag_quest_sequences, maxlen=max_sequence_length)

In [65]:
#concat the test Pretrained Glove Word Embedding with Pos_tag
test_concated_docs = test_tag_doc_sequences_padded + x_test_documents
test_concated_quest = test_tag_doc_sequences_padded + x_test_questions

In [66]:
model.save('model_glove_POS_epoch10.h5')

#test with the concatenated test data with POS Tag test data
# Make predictions
y_pred = model.predict([test_concated_docs, test_concated_quest])

# Convert predicted labels to class indices (assuming one-hot encoding)
y_pred_indices = np.argmax(y_pred, axis=-1)
y_true_indices = np.argmax(y_test_labels, axis=-1)

# Flatten the indices for computing metrics
y_pred_indices_flat = y_pred_indices.flatten()
y_true_indices_flat = y_true_indices.flatten()


print(classification_report(y_pred_indices_flat,y_true_indices_flat))

# Compute precision, recall, and F1-score
precision_withPOS = precision_score(y_true_indices_flat, y_pred_indices_flat, average='weighted')
recall_withPOS = recall_score(y_true_indices_flat, y_pred_indices_flat, average='weighted')
f1_withPOS = f1_score(y_true_indices_flat, y_pred_indices_flat, average='weighted')

 #Print the metrics
print('Precision:', precision_withPOS)
print('Recall:', recall_withPOS)
print('F1-score:', f1_withPOS)

10/10 [==============================] - 4s 383ms/step
              precision    recall  f1-score   support

           0       1.00      0.15      0.26     13020
           1       0.26      0.99      0.41        73
           2       0.33      0.37      0.35      7558
           3       0.35      0.75      0.48      8449

    accuracy                           0.38     29100
   macro avg       0.48      0.56      0.37     29100
weighted avg       0.63      0.38      0.35     29100

Precision: 0.5996710614931302
Recall: 0.38470790378006875
F1-score: 0.42235677889687684


#### Glove+POS+TF_IDF

In [67]:
#Convert TF_IDF to sequence
train_TF_IDF = train_joined_df[train_joined_df['Label'] == 1]['Merged Tokens']

tag_tf_sequences = tokenizer.texts_to_sequences(train_TF_IDF)

In [68]:
# Pad the sequences to a maximum length of 100 
# make all the sequences to same length
train_tf_sequences_padded = pad_sequences(tag_tf_sequences, maxlen=max_sequence_length)

In [69]:
#concat the test Pretrained Glove Word Embedding with Pos_tag and TF IDF
train_concated_docs_all = train_tag_doc_sequences_padded + x_train_documents + train_tf_sequences_padded
train_concated_quest_all = train_tag_doc_sequences_padded + x_train_questions + train_tf_sequences_padded

In [70]:
# Train the model
model.fit(x=[train_concated_quest_all, train_concated_docs_all], y=y_train_labels, epochs=10)

Epoch 1/10
33/33 [==============================] - 54s 2s/step - loss: 0.5731 - accuracy: 0.7222
Epoch 2/10
33/33 [==============================] - 54s 2s/step - loss: 0.5648 - accuracy: 0.7242
Epoch 3/10
33/33 [==============================] - 54s 2s/step - loss: 0.5622 - accuracy: 0.7251
Epoch 4/10
33/33 [==============================] - 54s 2s/step - loss: 0.5562 - accuracy: 0.7304
Epoch 5/10
33/33 [==============================] - 58s 2s/step - loss: 0.5514 - accuracy: 0.7342
Epoch 6/10
33/33 [==============================] - 54s 2s/step - loss: 0.5454 - accuracy: 0.7387
Epoch 7/10
33/33 [==============================] - 53s 2s/step - loss: 0.5428 - accuracy: 0.7429
Epoch 8/10
33/33 [==============================] - 55s 2s/step - loss: 0.5359 - accuracy: 0.7478
Epoch 9/10
33/33 [==============================] - 54s 2s/step - loss: 0.5270 - accuracy: 0.7528
Epoch 10/10
33/33 [==============================] - 54s 2s/step - loss: 0.5215 - accuracy: 0.7567


In [71]:
#Convert TF_IDF to sequence
test_TF_IDF = test_joined_df[test_joined_df['Label'] == 1]['Merged Tokens']

test_tag_tf_sequences = tokenizer.texts_to_sequences(test_TF_IDF)

In [72]:
# Pad the sequences to a maximum length of 100 
# make all the sequences to same length
test_tf_sequences_padded = pad_sequences(test_tag_tf_sequences, maxlen=max_sequence_length)

In [73]:
#concat the test Pretrained Glove Word Embedding with Pos_tag and TF IDF
test_concated_docs_all = test_tag_doc_sequences_padded + x_test_documents + test_tf_sequences_padded
test_concated_quest_all = test_tag_doc_sequences_padded + x_test_questions + test_tf_sequences_padded

In [74]:
model.save('model_glove_POS_TFIDF_epoch10.h5')

#test with the concatenated test data with POS Tag test data
# Make predictions
y_pred = model.predict([test_concated_docs_all, test_concated_quest_all])

# Convert predicted labels to class indices (assuming one-hot encoding)
y_pred_indices = np.argmax(y_pred, axis=-1)
y_true_indices = np.argmax(y_test_labels, axis=-1)

# Flatten the indices for computing metrics
y_pred_indices_flat = y_pred_indices.flatten()
y_true_indices_flat = y_true_indices.flatten()


print(classification_report(y_pred_indices_flat,y_true_indices_flat))

# Compute precision, recall, and F1-score
precision_withall = precision_score(y_true_indices_flat, y_pred_indices_flat, average='weighted')
recall_withall= recall_score(y_true_indices_flat, y_pred_indices_flat, average='weighted')
f1_withall = f1_score(y_true_indices_flat, y_pred_indices_flat, average='weighted')

 #Print the metrics
print('Precision:', precision_withall)
print('Recall:', recall_withall)
print('F1-score:', f1_withall)

10/10 [==============================] - 4s 381ms/step
              precision    recall  f1-score   support

           0       1.00      0.14      0.24     14533
           1       0.18      0.41      0.25       124
           2       0.42      0.32      0.36     11413
           3       0.12      0.71      0.20      3030

    accuracy                           0.27     29100
   macro avg       0.43      0.39      0.26     29100
weighted avg       0.68      0.27      0.28     29100

Precision: 0.5537461592134061
Recall: 0.2683505154639175
F1-score: 0.2531859135703821


In [75]:
from tabulate import tabulate
Embedding=['Glove','Glove+POS','Glove+POS+TF_IDF']
precision = [precision_cosattn,precision_withPOS,precision_withall]
recall = [recall_cosattn,recall_withPOS,recall_withall]
f1 = [f1_cosattn,f1_withPOS,f1_withall]

table_data = zip(Embedding, precision, recall, f1)
headers = ["Embedding","Precision", "Recall", "F1 Score"]
table = tabulate(table_data, headers=headers, tablefmt="fancy_grid")

print(table)

╒══════════════════╤═════════════╤══════════╤════════════╕
│ Embedding        │   Precision │   Recall │   F1 Score │
╞══════════════════╪═════════════╪══════════╪════════════╡
│ Glove            │    0.723428 │ 0.709897 │   0.620342 │
├──────────────────┼─────────────┼──────────┼────────────┤
│ Glove+POS        │    0.599671 │ 0.384708 │   0.422357 │
├──────────────────┼─────────────┼──────────┼────────────┤
│ Glove+POS+TF_IDF │    0.553746 │ 0.268351 │   0.253186 │
╘══════════════════╧═════════════╧══════════╧════════════╛


###3.2. Attention Ablation Study
(You can add as many code blocks and text blocks as you need. However, YOU SHOULD NOT MODIFY the section title) 

In [76]:
# Apply attention mechanism to the document LSTM output

attention = scaled_dot_product_attention([question_lstm, document_lstm])

# Reshape attention weights to match LSTM output shape
attention = Reshape((-1, max_sequence_length))(attention)

# Permute attention weights to match LSTM output shape
attention = Lambda(lambda x: K.permute_dimensions(x, (0, 2, 1)))(attention)

# Apply attention weights to LSTM output
weighted_representation = Dot(axes=(2, 1))([attention, document_lstm])

# Concatenate attention-weighted representation with LSTM output
concatenated_representation = Concatenate()([document_lstm, weighted_representation])

# Output layer for token prediction
output = TimeDistributed(Dense(4, activation='softmax'))(concatenated_representation)

# Create the model
model = keras.Model(inputs=[question_input, document_input], outputs=output)

# Define the loss function
loss_fn = keras.losses.CategoricalCrossentropy()

# Compile the model
model.compile(optimizer='adam', loss=loss_fn, metrics=['accuracy'])

# Print the model summary
model.summary()

# Train the model
model.fit(x=[x_train_questions, x_train_documents], y=y_train_labels, epochs=10)
model.save('model_sdattn_epoch10.h5')

# Make predictions
y_pred = model.predict([x_test_questions, x_test_documents])

# Convert predicted labels to class indices (assuming one-hot encoding)
y_pred_indices = np.argmax(y_pred, axis=-1)
y_true_indices = np.argmax(y_test_labels, axis=-1)

# Flatten the indices for computing metrics
y_pred_indices_flat = y_pred_indices.flatten()
y_true_indices_flat = y_true_indices.flatten()


print(classification_report(y_pred_indices_flat,y_true_indices_flat))

# Compute precision, recall, and F1-score
precision_sdattn = precision_score(y_true_indices_flat, y_pred_indices_flat, average='weighted')
recall_sdattn = recall_score(y_true_indices_flat, y_pred_indices_flat, average='weighted')
f1_sdattn = f1_score(y_true_indices_flat, y_pred_indices_flat, average='weighted')

 #Print the metrics
print('Precision:', precision_sdattn)
print('Recall:', recall_sdattn)
print('F1-score:', f1_sdattn)

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 document_input (InputLayer)    [(None, 100)]        0           []                               
                                                                                                  
 question_input (InputLayer)    [(None, 100)]        0           []                               
                                                                                                  
 embedding (Embedding)          (None, 100, 100)     40000100    ['question_input[0][0]',         
                                                                  'document_input[0][0]']         
                                                                                                  
 bidirectional_2 (Bidirectional  (None, 100, 256)    234496      ['embedding[1][0]']        

In [77]:
# Apply attention mechanism to the document LSTM output
attention = dot_product_attention([question_lstm, document_lstm])

# Reshape attention weights to match LSTM output shape
attention = Reshape((-1, max_sequence_length))(attention)

# Permute attention weights to match LSTM output shape
attention = Lambda(lambda x: K.permute_dimensions(x, (0, 2, 1)))(attention)

# Apply attention weights to LSTM output
weighted_representation = Dot(axes=(2, 1))([attention, document_lstm])

# Concatenate attention-weighted representation with LSTM output
concatenated_representation = Concatenate()([document_lstm, weighted_representation])

# Output layer for token prediction
output = TimeDistributed(Dense(4, activation='softmax'))(concatenated_representation)

# Create the model
model = keras.Model(inputs=[question_input, document_input], outputs=output)

# Define the loss function
loss_fn = keras.losses.CategoricalCrossentropy()

# Compile the model
model.compile(optimizer='adam', loss=loss_fn, metrics=['accuracy'])

# Print the model summary
model.summary()

# Train the model
model.fit(x=[x_train_questions, x_train_documents], y=y_train_labels, epochs=10)
model.save('model_dattn_epoch10.h5')

# Make predictions
y_pred = model.predict([x_test_questions, x_test_documents])

# Convert predicted labels to class indices (assuming one-hot encoding)
y_pred_indices = np.argmax(y_pred, axis=-1)
y_true_indices = np.argmax(y_test_labels, axis=-1)

# Flatten the indices for computing metrics
y_pred_indices_flat = y_pred_indices.flatten()
y_true_indices_flat = y_true_indices.flatten()


print(classification_report(y_pred_indices_flat,y_true_indices_flat))

# Compute precision, recall, and F1-score
precision_dattn = precision_score(y_true_indices_flat, y_pred_indices_flat, average='weighted')
recall_dattn = recall_score(y_true_indices_flat, y_pred_indices_flat, average='weighted')
f1_dattn = f1_score(y_true_indices_flat, y_pred_indices_flat, average='weighted')

 #Print the metrics
print('Precision:', precision_dattn)
print('Recall:', recall_dattn)
print('F1-score:', f1_dattn)

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 document_input (InputLayer)    [(None, 100)]        0           []                               
                                                                                                  
 question_input (InputLayer)    [(None, 100)]        0           []                               
                                                                                                  
 embedding (Embedding)          (None, 100, 100)     40000100    ['question_input[0][0]',         
                                                                  'document_input[0][0]']         
                                                                                                  
 bidirectional_2 (Bidirectional  (None, 100, 256)    234496      ['embedding[1][0]']        

In [78]:
from tabulate import tabulate
Attention=['Dot-product','Scale-Dot-Product','Cosine-Product']
precision = [precision_dattn,precision_sdattn,precision_cosattn]
recall = [recall_dattn,recall_sdattn,recall_cosattn]
f1 = [f1_dattn,f1_sdattn,f1_cosattn]

table_data = zip(Attention, precision, recall, f1)
headers = ["Attention", "Precision", "Recall", "F1 Score"]
table = tabulate(table_data, headers=headers, tablefmt="fancy_grid")

print(table)

╒═══════════════════╤═════════════╤══════════╤════════════╕
│ Attention         │   Precision │   Recall │   F1 Score │
╞═══════════════════╪═════════════╪══════════╪════════════╡
│ Dot-product       │    0.651233 │ 0.675395 │   0.655722 │
├───────────────────┼─────────────┼──────────┼────────────┤
│ Scale-Dot-Product │    0.680601 │ 0.708282 │   0.66464  │
├───────────────────┼─────────────┼──────────┼────────────┤
│ Cosine-Product    │    0.723428 │ 0.709897 │   0.620342 │
╘═══════════════════╧═════════════╧══════════╧════════════╛


###3.3. Hyper Parameter Testing
(You can add as many code blocks and text blocks as you need. However, YOU SHOULD NOT MODIFY the section title) 

In [79]:
# Train the model Epoch = 5
model.fit(x=[x_train_questions, x_train_documents], y=y_train_labels, epochs=5)

model.save('model_epoch5.h5') 

y_pred = model.predict([x_test_questions, x_test_documents])

# Convert predicted labels to class indices (assuming one-hot encoding)
y_pred_indices = np.argmax(y_pred, axis=-1)
y_true_indices = np.argmax(y_test_labels, axis=-1)

# Flatten the indices for computing metrics
y_pred_indices_flat = y_pred_indices.flatten()
y_true_indices_flat = y_true_indices.flatten()


print(classification_report(y_pred_indices_flat,y_true_indices_flat))
# Compute precision, recall, and F1-score
precision_epoch5 = precision_score(y_true_indices_flat, y_pred_indices_flat, average='weighted')
recall_epoch5 = recall_score(y_true_indices_flat, y_pred_indices_flat, average='weighted')
f1_epoch5= f1_score(y_true_indices_flat, y_pred_indices_flat, average='weighted')

 #Print the metrics
print('Precision:', precision_epoch5)
print('Recall:', recall_epoch5)
print('F1-score:', f1_epoch5)

Epoch 1/5
33/33 [==============================] - 54s 2s/step - loss: 0.4948 - accuracy: 0.7684
Epoch 2/5
33/33 [==============================] - 55s 2s/step - loss: 0.4906 - accuracy: 0.7703
Epoch 3/5
33/33 [==============================] - 54s 2s/step - loss: 0.4903 - accuracy: 0.7707
Epoch 4/5
33/33 [==============================] - 55s 2s/step - loss: 0.4839 - accuracy: 0.7739
Epoch 5/5
10/10 [==============================] - 5s 505ms/step
              precision    recall  f1-score   support

           0       0.93      0.91      0.92      2024
           1       0.80      0.93      0.86       241
           2       0.22      0.44      0.30      4363
           3       0.86      0.70      0.77     22472

    accuracy                           0.68     29100
   macro avg       0.70      0.74      0.71     29100
weighted avg       0.77      0.68      0.71     29100

Precision: 0.6389720614261446
Recall: 0.6765635738831615
F1-score: 0.6417206849659873


In [80]:
# Train the model Epoch = 10
model.fit(x=[x_train_questions, x_train_documents], y=y_train_labels, epochs=10)

model.save('model_epoch10.h5') 

y_pred = model.predict([x_test_questions, x_test_documents])

# Convert predicted labels to class indices (assuming one-hot encoding)
y_pred_indices = np.argmax(y_pred, axis=-1)
y_true_indices = np.argmax(y_test_labels, axis=-1)

# Flatten the indices for computing metrics
y_pred_indices_flat = y_pred_indices.flatten()
y_true_indices_flat = y_true_indices.flatten()


print(classification_report(y_pred_indices_flat,y_true_indices_flat))
# Compute precision, recall, and F1-score
precision_epoch10 = precision_score(y_true_indices_flat, y_pred_indices_flat, average='weighted')
recall_epoch10 = recall_score(y_true_indices_flat, y_pred_indices_flat, average='weighted')
f1_epoch10 = f1_score(y_true_indices_flat, y_pred_indices_flat, average='weighted')

 #Print the metrics
print('Precision:', precision_epoch10)
print('Recall:', recall_epoch10)
print('F1-score:', f1_epoch10)

Epoch 1/10
33/33 [==============================] - 54s 2s/step - loss: 0.4715 - accuracy: 0.7780
Epoch 2/10
33/33 [==============================] - 55s 2s/step - loss: 0.4691 - accuracy: 0.7788
Epoch 3/10
33/33 [==============================] - 55s 2s/step - loss: 0.4633 - accuracy: 0.7800
Epoch 4/10
33/33 [==============================] - 53s 2s/step - loss: 0.4580 - accuracy: 0.7810
Epoch 5/10
33/33 [==============================] - 56s 2s/step - loss: 0.4523 - accuracy: 0.7845
Epoch 6/10
33/33 [==============================] - 56s 2s/step - loss: 0.4446 - accuracy: 0.7861
Epoch 7/10
33/33 [==============================] - 54s 2s/step - loss: 0.4389 - accuracy: 0.7898
Epoch 8/10
33/33 [==============================] - 55s 2s/step - loss: 0.4283 - accuracy: 0.7925
Epoch 9/10
33/33 [==============================] - 55s 2s/step - loss: 0.4239 - accuracy: 0.7946
Epoch 10/10
10/10 [==============================] - 6s 562ms/step
              precision    recall  f1-score   suppo

In [82]:
# Train the model Epoch = 15
model.fit(x=[x_train_questions, x_train_documents], y=y_train_labels, epochs=15)

model.save('model_epoch15.h5') 

y_pred = model.predict([x_test_questions, x_test_documents])

# Convert predicted labels to class indices (assuming one-hot encoding)
y_pred_indices = np.argmax(y_pred, axis=-1)
y_true_indices = np.argmax(y_test_labels, axis=-1)

# Flatten the indices for computing metrics
y_pred_indices_flat = y_pred_indices.flatten()
y_true_indices_flat = y_true_indices.flatten()


print(classification_report(y_pred_indices_flat,y_true_indices_flat))
# Compute precision, recall, and F1-score
precision_epoch15 = precision_score(y_true_indices_flat, y_pred_indices_flat, average='weighted')
recall_epoch15 = recall_score(y_true_indices_flat, y_pred_indices_flat, average='weighted')
f1_epoch15 = f1_score(y_true_indices_flat, y_pred_indices_flat, average='weighted')

 #Print the metrics
print('Precision:', precision_epoch15)
print('Recall:', recall_epoch15)
print('F1-score:', f1_epoch15)

Epoch 1/15
33/33 [==============================] - 70s 2s/step - loss: 0.2695 - accuracy: 0.8750
Epoch 2/15
33/33 [==============================] - 65s 2s/step - loss: 0.2596 - accuracy: 0.8802
Epoch 3/15
33/33 [==============================] - 56s 2s/step - loss: 0.2540 - accuracy: 0.8828
Epoch 4/15
33/33 [==============================] - 61s 2s/step - loss: 0.2489 - accuracy: 0.8849
Epoch 5/15
33/33 [==============================] - 53s 2s/step - loss: 0.2387 - accuracy: 0.8905
Epoch 6/15
33/33 [==============================] - 57s 2s/step - loss: 0.2279 - accuracy: 0.8963
Epoch 7/15
33/33 [==============================] - 55s 2s/step - loss: 0.2183 - accuracy: 0.9007
Epoch 8/15
33/33 [==============================] - 55s 2s/step - loss: 0.2172 - accuracy: 0.9031
Epoch 9/15
33/33 [==============================] - 55s 2s/step - loss: 0.2037 - accuracy: 0.9093
Epoch 10/15
33/33 [==============================] - 55s 2s/step - loss: 0.2036 - accuracy: 0.9084
Epoch 11/15
33/33 [

In [83]:
# Train the model Epoch 20
model.fit(x=[x_train_questions, x_train_documents], y=y_train_labels, epochs=20)
model.save('model_epoch20.h5')

Epoch 1/20
33/33 [==============================] - 55s 2s/step - loss: 0.1675 - accuracy: 0.9246
Epoch 2/20
33/33 [==============================] - 56s 2s/step - loss: 0.1620 - accuracy: 0.9267
Epoch 3/20
33/33 [==============================] - 55s 2s/step - loss: 0.1557 - accuracy: 0.9298
Epoch 4/20
33/33 [==============================] - 55s 2s/step - loss: 0.1558 - accuracy: 0.9303
Epoch 5/20
33/33 [==============================] - 55s 2s/step - loss: 0.1564 - accuracy: 0.9295
Epoch 6/20
33/33 [==============================] - 54s 2s/step - loss: 0.1499 - accuracy: 0.9314
Epoch 7/20
33/33 [==============================] - 54s 2s/step - loss: 0.1477 - accuracy: 0.9323
Epoch 8/20
33/33 [==============================] - 55s 2s/step - loss: 0.1442 - accuracy: 0.9331
Epoch 9/20
33/33 [==============================] - 55s 2s/step - loss: 0.1390 - accuracy: 0.9363
Epoch 10/20
33/33 [==============================] - 57s 2s/step - loss: 0.1379 - accuracy: 0.9358
Epoch 11/20
33/33 [

In [84]:
y_pred = model.predict([x_test_questions, x_test_documents])

# Convert predicted labels to class indices (assuming one-hot encoding)
y_pred_indices = np.argmax(y_pred, axis=-1)
y_true_indices = np.argmax(y_test_labels, axis=-1)

# Flatten the indices for computing metrics
y_pred_indices_flat = y_pred_indices.flatten()
y_true_indices_flat = y_true_indices.flatten()

print(classification_report(y_pred_indices_flat,y_true_indices_flat))

# Compute precision, recall, and F1-score
precision_epoch20 = precision_score(y_true_indices_flat, y_pred_indices_flat, average='weighted')
recall_epoch20 = recall_score(y_true_indices_flat, y_pred_indices_flat, average='weighted')
f1_epoch20 = f1_score(y_true_indices_flat, y_pred_indices_flat, average='weighted')

 #Print the metrics
print('Precision:', precision_epoch20)
print('Recall:', recall_epoch20)
print('F1-score:', f1_epoch20)

10/10 [==============================] - 4s 376ms/step
              precision    recall  f1-score   support

           0       0.89      0.91      0.90      1928
           1       0.73      0.86      0.79       237
           2       0.34      0.39      0.36      7479
           3       0.75      0.70      0.72     19456

    accuracy                           0.64     29100
   macro avg       0.68      0.72      0.70     29100
weighted avg       0.65      0.64      0.64     29100

Precision: 0.6256916293784779
Recall: 0.6373539518900344
F1-score: 0.6304183384972283


In [85]:
# Train the model Epoch 25
model.fit(x=[x_train_questions, x_train_documents], y=y_train_labels, epochs=25)
model.save('model_epoch25.h5')

Epoch 1/25
33/33 [==============================] - 56s 2s/step - loss: 0.1197 - accuracy: 0.9403
Epoch 2/25
33/33 [==============================] - 52s 2s/step - loss: 0.1193 - accuracy: 0.9408
Epoch 3/25
33/33 [==============================] - 54s 2s/step - loss: 0.1168 - accuracy: 0.9415
Epoch 4/25
33/33 [==============================] - 55s 2s/step - loss: 0.1137 - accuracy: 0.9420
Epoch 5/25
33/33 [==============================] - 56s 2s/step - loss: 0.1168 - accuracy: 0.9420
Epoch 6/25
33/33 [==============================] - 56s 2s/step - loss: 0.1147 - accuracy: 0.9418
Epoch 7/25
33/33 [==============================] - 54s 2s/step - loss: 0.1123 - accuracy: 0.9425
Epoch 8/25
33/33 [==============================] - 55s 2s/step - loss: 0.1124 - accuracy: 0.9419
Epoch 9/25
33/33 [==============================] - 56s 2s/step - loss: 0.1079 - accuracy: 0.9436
Epoch 10/25
33/33 [==============================] - 55s 2s/step - loss: 0.1084 - accuracy: 0.9423
Epoch 11/25
33/33 [

In [86]:
y_pred = model.predict([x_test_questions, x_test_documents])

# Convert predicted labels to class indices (assuming one-hot encoding)
y_pred_indices = np.argmax(y_pred, axis=-1)
y_true_indices = np.argmax(y_test_labels, axis=-1)

# Flatten the indices for computing metrics
y_pred_indices_flat = y_pred_indices.flatten()
y_true_indices_flat = y_true_indices.flatten()


print(classification_report(y_pred_indices_flat,y_true_indices_flat))
# Compute precision, recall, and F1-score
precision_epoch25 = precision_score(y_true_indices_flat, y_pred_indices_flat, average='weighted')
recall_epoch25 = recall_score(y_true_indices_flat, y_pred_indices_flat, average='weighted')
f1_epoch25 = f1_score(y_true_indices_flat, y_pred_indices_flat, average='weighted')

 #Print the metrics
print('Precision:', precision_epoch25)
print('Recall:', recall_epoch25)
print('F1-score:', f1_epoch25)

10/10 [==============================] - 4s 391ms/step
              precision    recall  f1-score   support

           0       0.88      0.92      0.90      1878
           1       0.78      0.79      0.78       276
           2       0.38      0.39      0.38      8538
           3       0.71      0.70      0.71     18408

    accuracy                           0.63     29100
   macro avg       0.69      0.70      0.69     29100
weighted avg       0.63      0.63      0.63     29100

Precision: 0.6259294494905437
Recall: 0.6261855670103093
F1-score: 0.6260032420615026


In [87]:
from tabulate import tabulate
epochs=[5,10,15,20,25]
precision = [precision_epoch5,precision_epoch10,precision_epoch15,precision_epoch20,precision_epoch25]
recall = [recall_epoch5,recall_epoch10,recall_epoch15,recall_epoch20,recall_epoch25]
f1 = [f1_epoch5,f1_epoch10,f1_epoch15,f1_epoch20,f1_epoch25]

table = [[''] + ['epoch ' + str(i) for i in epochs], 
         ['Precision'] + precision,
         ['Recall'] + recall,
         ['F1'] + f1]

print(tabulate(table, headers='firstrow', tablefmt='fancy_grid'))


╒═══════════╤═══════════╤════════════╤════════════╤════════════╤════════════╕
│           │   epoch 5 │   epoch 10 │   epoch 15 │   epoch 20 │   epoch 25 │
╞═══════════╪═══════════╪════════════╪════════════╪════════════╪════════════╡
│ Precision │  0.638972 │   0.638842 │   0.626441 │   0.625692 │   0.625929 │
├───────────┼───────────┼────────────┼────────────┼────────────┼────────────┤
│ Recall    │  0.676564 │   0.660481 │   0.639347 │   0.637354 │   0.626186 │
├───────────┼───────────┼────────────┼────────────┼────────────┼────────────┤
│ F1        │  0.641721 │   0.634887 │   0.631503 │   0.630418 │   0.626003 │
╘═══════════╧═══════════╧════════════╧════════════╧════════════╧════════════╛
